# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [57]:
def priest(sex, age, resp_rate, O2_sat, HR, sys_BP, temp, alertness, insp_O2, per_status):
  """
  The function requires the following input parameters:
  -Sex (Gender assigned at birth)
  -Age in years
  -Respiratory rate in breaths per minute
  -Oxygen saturation as a percent between 0 and 1
  -Heart rate in beats per minute
  -Systolic BP in mmHg
  -Temperature in degrees C
  -Alertness as a string description
  -Inspired Oxygen as as string description
  -Performance Status as a string description

  Then, it adheres to the algorithm listed on and to calculate a risk percentage and return it as a numeric value between 0 and 1.

  >>> priest('male', 56, 11, 0.91, 56, 102, 36.9, 'alert', 'air', 'limited self-care')
  0.29
  >>> priest('male', 55, 18, 0.90, 90, 160, 38.5, 'confused or not alert', 'air', 'bed/chair bound, no self-care')
  0.46
  >>> priest('female', 80, 26, 0.88, 140, 180, 38.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
  0.55
  """

  points = 0
  "sex check"
  if sex.upper() == 'MALE':
    points += 1
  elif sex.upper() == 'FEMALE':
    pass
  else:
    return None

  "age check"
  if 16 <= age <= 49:
    pass
  elif 50 <= age <= 65:
    points += 2
  elif 66 <= age <= 80:
    points += 3
  elif age > 80:
    points += 4
  else:
    return None

  "Assessing resp_rate(bpm)"
  if resp_rate < 9 or resp_rate > 24:
    points += 3
  elif 9 <= resp_rate <= 11:
    points += 1
  elif 12 <= resp_rate <= 20:
    pass
  elif 21 <= resp_rate <= 24:
    points += 2
  else:
    return None

  "Evaluating O2_sat"
  if O2_sat > 0.95:
    pass
  elif 0.94 >= O2_sat >= 0.95:
    points += 1
  elif 0.92 <= O2_sat <= 0.93:
    points += 2
  elif O2_sat < 0.92:
    points += 3
  else:
    return None

  "Heart Rate"
  if HR < 41 or HR > 130:
    points += 3
  elif 41 <= HR <= 50:
    points += 1
  elif 51 <= HR <= 90:
    pass
  elif 91<= HR <= 110:
    points += 1
  elif 111 <= HR <= 130:
    points += 2
  else:
    return None

  "Systolic Blood Pressure"
  if sys_BP < 91 or sys_BP > 219:
    points += 3
  elif 91 <= sys_BP <= 100:
    points += 2
  elif 101 <= sys_BP <= 110:
    points += 1
  elif 111 <= sys_BP <= 219:
    pass
  else:
    return None

  "Temperature"
  if temp < 35.1:
    points += 3
  elif 35.1 <= temp <= 36.0:
    points += 1
  elif 36.1 <= temp <= 38.0:
    pass
  elif 38.1 <= temp <= 39.0:
    points += 1
  elif temp > 39.0:
    points += 2
  else:
    return None

  "alertness"
  alertness = alertness.lower()
  if alertness == 'alert':
    pass
  elif alertness == 'confused or not alert':
    points += 3
  else:
    return None

  "Inspired Oxygen"
  insp_O2 = insp_O2.lower()
  if insp_O2 == 'air':
    pass
  elif insp_O2 == 'supplemental oxygen':
    points += 2
  else:
    return None

  "Performance Status"
  per_status = per_status.lower()
  if per_status == 'unrestricted normal activity':
    pass
  elif per_status == 'limited strenuous activity, can do light activity':
    points += 1
  elif per_status == 'limited activity, can self-care':
    points += 2
  elif per_status == 'limited self-care':
    points += 3
  elif per_status == 'bed/chair bound, no self-care':
    points += 4
  else:
    return None


  "Probability Score"
  if points < 2:
    return 0.01
  elif points < 4:
    return 0.02
  elif points < 5:
    return 0.03
  elif points < 6:
    return 0.09
  elif points < 7:
    return 0.15
  elif points < 8:
    return 0.18
  elif points < 9:
    return 0.22
  elif points < 10:
    return 0.26
  elif points < 12:
    return 0.29
  elif points < 13:
    return 0.34
  elif points < 14:
    return 0.38
  elif points < 15:
    return 0.46
  elif points < 16:
    return 0.47
  elif points < 17:
    return 0.49
  elif points <= 25:
    return 0.55
  else:
    return 0.99

In [58]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 56, 11, 0.91, 56, 102, 36.9, 'alert', 'air', 'limited self-care')
Expecting:
    0.29
ok
Trying:
    priest('male', 55, 18, 0.90, 90, 160, 38.5, 'confused or not alert', 'air', 'bed/chair bound, no self-care')
Expecting:
    0.46
ok
Trying:
    priest('female', 80, 26, 0.88, 140, 180, 38.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.55
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [59]:
import requests
def find_hospital(age, sex, risk):

  """
  The function uses three parameters:age, sex, risk. Which makes a request to the REST web service.

  >>> find_hospital('95', 'male', '.79')
  'Wesley Woods Geriatric Hospital'

  >>> find_hospital('12', 'female', '.59')
  'Hughes Spalding Childrens Hospital'

  >>> find_hospital('93', 'female', '.99')
  'Wesley Woods Geriatric Hospital'

  """

  url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?"

  #Preparing parameters as a dictionary
  params = {'age':age, 'sex':sex.lower(), 'risk_pct':risk}

  try:
    response = requests.get(url, params = params)
    #check if request was successful
    if response.status_code == 200:
      re=response.json()
      return (re['hospital'])
  except Exception as e:
      return None

In [60]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital('95', 'male', '.79')
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital('12', 'female', '.59')
Expecting:
    'Hughes Spalding Childrens Hospital'
ok
Trying:
    find_hospital('93', 'female', '.99')
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [61]:
import requests
def get_address(hospital_name):
  """
  This function gives the hospital.address for the identified hospital.
  Addresses for the hospital is found from the url: https://www.officialusa.com/stateguides/health/hospitals/georgia.html

  :params hospital_name: The name of hospital.
  :return: The address of the hospital, or None if not found.

  >>> get_address('WELLSTAR PAULDING HOSPITAL')
  '600 WEST MEMORIAL DRIVE'

  >>> get_address('REGENCY HOSPITAL OF SOUTH ATLANTA')
  '1170 CLEVELAND AVE, 4TH FLOOR'

  >>> get_address('HAMILTON MEDICAL CENTER')
  '1200 MEMORIAL DRIVE'

  >>> get_address('Nonexistent Hospital')
  None
  """

  url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
  response = requests.get(url)

  if response is not None and response.status_code == 200:
      hospital_data = response.json()
      hospital_name = hospital_name.upper()

      if hospital_name in hospital_data:
        return hospital_data[hospital_name]['ADDRESS']
      else:
        print (None)
  elif response is None:
    return None
  return None

In [62]:
doctest.run_docstring_examples(get_address,globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('WELLSTAR PAULDING HOSPITAL')
Expecting:
    '600 WEST MEMORIAL DRIVE'
ok
Trying:
    get_address('REGENCY HOSPITAL OF SOUTH ATLANTA')
Expecting:
    '1170 CLEVELAND AVE, 4TH FLOOR'
ok
Trying:
    get_address('HAMILTON MEDICAL CENTER')
Expecting:
    '1200 MEMORIAL DRIVE'
ok
Trying:
    get_address('Nonexistent Hospital')
Expecting:
    None
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [63]:
import requests

def process_people(file_url):
    """
    Calculating risk scores and assign hospitals by using people.psv file.

    Parameters:
    file_url(str): URL to be processed.

    Returns:
    list: A list of dictionaries containing patient details and assigned hospital and address.
    """
    people_results = []
    response = requests.get(file_url)

    if response.status_code == 200:
        content = response.text
        lines = content.splitlines()

        for line in lines[1:]:  # Skip header line
            fields = line.split('|')
            if len(fields) < 11:
                print(f"Skipping invalid line: {line}")
                continue  # Skip invalid lines

            # Extracting data
            details = [
                fields[1].strip(),
                int(fields[2]),
                float(fields[3]),
                float(fields[4]),
                float(fields[5]),
                float(fields[6]),
                float(fields[7]),
                fields[8].strip(),
                fields[9].strip(),
                fields[10].strip()
            ]

            # Add patient to list
            people_results.append({fields[0]: details})  # Use fields[0] for patient ID

        print(f"Processed {len(people_results)} patients")  # Check how many patients were processed
    else:
        print(f"Error fetching file: {response.status_code}")

    return people_results

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [64]:
def compare_data(local, online):
    """
    Compares local patient data with online patient data.
    :param local: List of local patient data
    :param online: Dictionary containing online patient data
    """
    matches = 0
    mismatches = 0
    for local_patient in local:
        for patient_id, local_details in local_patient.items():
            online_details = online.get(patient_id)

            if online_details:
              online_details = online_details[:10]
                # Check if local and online data match
              if local_details == online_details:
                    print(f"Patient {patient_id} details match.")
                    matches += 1
              else:
                    print(f"Patient {patient_id} details do NOT match.")
                    print(f"Local details: {local_details}")
                    print(f"Online details: {online_details}")
                    mismatches += 1
            else:
                print(f"Patient {patient_id} not found in online data.")
                mismatches += 1
people_csv_url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
my_results = process_people(people_csv_url)

people_results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
people_results_response = requests.get(people_results_url)
people_results_json_data = people_results_response.json()

compare_data(local=my_results, online=people_results_json_data)

Processed 99 patients
Patient E9559 details match.
Patient E9385 details match.
Patient E3067 details match.
Patient E9422 details match.
Patient E8661 details match.
Patient E6235 details match.
Patient E4451 details match.
Patient E8433 details match.
Patient E7593 details match.
Patient E3296 details match.
Patient E4157 details match.
Patient E7702 details match.
Patient E8158 details match.
Patient E4795 details match.
Patient E8800 details match.
Patient E4855 details match.
Patient E9691 details match.
Patient E4535 details match.
Patient E2360 details match.
Patient E3447 details match.
Patient E8208 details match.
Patient E4428 details match.
Patient E2766 details match.
Patient E3888 details match.
Patient E7110 details match.
Patient E2668 details match.
Patient E8482 details match.
Patient E9619 details match.
Patient E1700 details match.
Patient E8585 details match.
Patient E8633 details match.
Patient E2117 details match.
Patient E2322 details match.
Patient E6912 details

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---